### Student Performance Prediction in Large Online Classes.
> * In this project, we build a model that uses video and learning platform interaction data to predict the score that a student will get.
> * Being built in the context of personalized learning. 


### 1. Raw Data Exploration and Training Data Generation
> * First, we explore the raw data we have and then extract features
the raw data to create a clean training, validation, and testing set.

> * We have data from 3 MOOCs offered through the Stanford's OpenEdx Platform

|OpenEdx Course | Enrollment Count|Year|Self Paced|
|---------------|-----------------|----|-----------|
| Mining Massive Datasets 1|4409|Fall 2015|No
| Mining Massive Datasets 2|9409|N/A|Yes
| Compilers|30485| Fall 2014| No
| Algorithms 1|16995|N/A|Yes
| Algorithms 2|1363|N/A|Yes
| Total|62, 661|N/A|N/A

For each of these courses we have the following Data:

1. Video interaction Logs
2. Activity Grades
3. Platform Interaction logs
4. Engagement Data 

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid")
from project_utilities import (
    read_data as read_csv
)
import dask.dataframe as dd

* We Begin by exploring the data in the tables associated with a single course.
* We'll look at the `Mining Massive Datasets` course (both the self-paced and normal)

### 1. Activity Grade Table

In [4]:
'''
Read activity grade data into memory, 
making sure to create 'clean' column names.
'''
mmds_one_grade = read_csv(
    './Data/MMDS/course-v1-ComputerScience+MMDS+Fall2016_ActivityGrade.csv'
)

mmds_two_grade = read_csv(
    './Data/MMDS/course-v1-ComputerScience+MMDS+SelfPaced_ActivityGrade.csv'
)

./Data/MMDS/course-v1-ComputerScience+MMDS+Fall2016_ActivityGrade.csv is not a valid filename
./Data/MMDS/course-v1-ComputerScience+MMDS+SelfPaced_ActivityGrade.csv is not a valid filename


* As can be seen above, the activity grade table includes so much info that isn't about grades.
* So, we remove that extra info.

In [3]:
mmds_one_grade = mmds_one_grade[mmds_one_grade.num_attempts >= 0]
mmds_two_grade = mmds_two_grade[mmds_two_grade.num_attempts >= 0]

In [4]:
mmds_one_grade.head(3)

,activity_grade_id,student_id,course_display_name,grade,max_grade,percent_grade,parts_correctness,answers,num_attempts,first_submit,last_submit,module_type,anon_screen_name,resource_display_name,module_id
52,67811991,308846,course-v1:ComputerScience+MMDS+Fall2016,1.0,5,20.0,--+--,"choice_8,choice_1,choice_17,choice_8",2,8/30/16 1:02,8/30/16 1:03,problem,79fe2526bb9262f9ae1c9a30f1b4add61a480a31,NaN,block-v1:ComputerScience+MMDS+Fall2016+type@pr...
55,67812024,308846,course-v1:ComputerScience+MMDS+Fall2016,1.0,4,25.0,--+-,"choice_5,choice_9,choice_1,choice_10",2,8/30/16 1:03,8/30/16 1:04,problem,79fe2526bb9262f9ae1c9a30f1b4add61a480a31,NaN,block-v1:ComputerScience+MMDS+Fall2016+type@pr...
61,67812105,308846,course-v1:ComputerScience+MMDS+Fall2016,1.0,4,25.0,--+-,"choice_8,choice_6,choice_1,choice_9,,incorrect...",1,8/30/16 1:05,8/30/16 1:05,problem,79fe2526bb9262f9ae1c9a30f1b4add61a480a31,NaN,block-v1:ComputerScience+MMDS+Fall2016+type@pr...


In [5]:
'''
Here we unify the data from the 
self paced version and the "normal"
version of the course.
'''
mmds_grade = mmds_one_grade.append(mmds_two_grade)

* The activity grade table will give us a value of $Y$, the response variable. 
* This value should be the average of the `percent grade` from the individual questions.
* Use this value because it accounts for people who did not complete the entire course.
* Such people would be excluded from final scores. Completion not measure of success (Kizlicec et al)

In [6]:
naive_agg = mmds_grade.groupby('anon_screen_name').agg({
    'num_attempts':np.mean, 
    'percent_grade':np.mean}
)
naive_agg.head(10)

,num_attempts,percent_grade
anon_screen_name,,
00f2937702328715338bee9301f70483bc3c157f,4.756757,60.945676
027bb45fa8011099227ead35dd024b39c7e72a49,1.666667,83.333333
02ce0cef19cc6cf469596eef28c1acf39dd4d306,1.592593,62.222222
02fc564414f4077f0912b9571440fafb79d1fea5,2.250000,87.500000
040ef94f6a47b427c044479bc5c9f687730887d6,5.000000,75.000000
0497b03a5ca2895fd367fb1121cef02700023a21,4.000000,100.000000
04c7e1cd9f9dcabea716d5ffc958ffaa6895726e,3.428571,96.428571
05d9253cdd24cca0a660f4904edaf58d5fb0c0fa,3.200000,95.000000
06807e60a37a3387407ce2b9575267f685ab37c1,2.500000,50.000000


* We see that this significantly slims down our table.
* Out of the $\approx 15000$ students enrolled in the two courses, 
  only 505 interacted with the quizzes/questions.
* My guess is that the number of people who interacted with vidoes is much 
  higher than this.
* Therefore, we may need to use these 505 students together with their 
  video watching behavior to impute the activity grade data fro the 
  other students in the class.
* However, we need to first figure out a good way of calculating $Y$

### 2. Video Interaction Logs.

In [5]:
mmds_one_video = read_csv(
    './Data/MMDS/course-v1-ComputerScience+MMDS+Fall2016_VideoInteraction.csv'
)
mmds_two_video = read_csv(
    './Data/MMDS/course-v1-ComputerScience+MMDS+SelfPaced_VideoInteraction.csv'
)

In [6]:
mmds_one_video.head(5)

,event_type,resource_display_name,video_current_time,video_speed,video_new_speed,video_old_speed,video_new_time,video_old_time,video_seek_type,video_code,time,course_display_name,quarter,anon_screen_name,video_id
0,'event_type','resource_display_name','video_current_time','video_speed','video_new_speed','video_old_speed','video_new_time','video_old_time','video_seek_type','video_code','time','course_display_name','quarter','anon_screen_name','video_id'
1,load_video,NaN,None,None,NaN,NaN,NaN,NaN,NaN,9bheA6knXdc,2016-12-04 21:23:10.881415,course-v1:ComputerScience+MMDS+Fall2016,winter2016,413458e047944547baafa5d894cde3e45d9d7bb4,961f99e0bb3643fda0ed20b882acd370
2,play_video,NaN,0.0019998550415,None,NaN,NaN,NaN,NaN,NaN,9bheA6knXdc,2016-12-04 21:23:15.790362,course-v1:ComputerScience+MMDS+Fall2016,winter2016,413458e047944547baafa5d894cde3e45d9d7bb4,961f99e0bb3643fda0ed20b882acd370
3,play_video,NaN,182.045444166,None,NaN,NaN,NaN,NaN,NaN,Y93J27otCWM,2016-12-04 21:24:19.420473,course-v1:ComputerScience+MMDS+Fall2016,winter2016,b284c767dab90551e79fe8d3ad18acfd37bf94f6,62bf333a7c2640f282c32be0ad9cd42e
4,pause_video,NaN,207.245429044,None,NaN,NaN,NaN,NaN,NaN,Y93J27otCWM,2016-12-04 21:24:44.860346,course-v1:ComputerScience+MMDS+Fall2016,winter2016,b284c767dab90551e79fe8d3ad18acfd37bf94f6,62bf333a7c2640f282c32be0ad9cd42e


* This table is records all the times that a learner interacts with the video component
  of the course. It catalogs the following events
  > 1. Playing a video
  > 2. Pausing a video
  > 3. `Seeking` a video, i.e going to a specific part of the video.
  > 4. Changing the video speed (*either speeding up or slowing down*)

In [18]:
mmds_video = mmds_one_video.append(mmds_two_video)

In [26]:
uniq_video = mmds_video.groupby(['anon_screen_name']).size().compute()

* As expected, out table shrinks. More importantly, however, is the fact 
  that it shrinks to a much larger value than the `Activity Grade` table.
* This means that we have quite a number of people who interact with the
  video component of the course without ever touching the questions.
* Our plan will be to use their video activity profile to impute their 
  `ActivityGrade` profile.

* At some point in our model building, we may decide to add granular 
  details about video viewing, i.e instead of just looking at the number of times
  a user interacted with th video, we may look at the number of times
  they perfomed specific actions like, play video, pause video e.t.c
* To obtain that info, we groupby event-type and name

In [27]:
granular_video = mmds_video.groupby(['anon_screen_name', 'event_type']).size().compute()

* We now have much granular info about video viewing behavior.
* We may at some point decide to even be more granular and further differentiate
  between the different kinds of `speed_change_video` and `seek_video` actions
* But let's put that aside for now.

### 3. Platform interaction Logs

In [2]:
mmds_one_platform = read_csv(
    './Data/MMDS/course-v1-ComputerScience+MMDS+SelfPaced_EventXtract.csv'
)
mmds_two_platform = read_csv(
    './Data/MMDS/course-v1-ComputerScience+MMDS+Fall2016_EventXtract.csv'
)

* These two tables also contain video interaction data. Since we already
  extracted taht data in $(2)$ above, we want to get rid of that bit of data.

In [3]:
mmds_one_platform.head(10)

,anon_screen_name,event_type,ip_country,time,quarter,course_display_name,resource_display_name,success,video_code,video_current_time,video_speed,video_old_time,video_new_time,video_seek_type,video_new_speed,video_old_speed,goto_from,goto_dest
0,'anon_screen_name','event_type','ip_country','time','quarter','course_display_name','resource_display_name','success','video_code','video_current_time','video_speed','video_old_time','video_new_time','video_seek_type','video_new_speed','video_old_speed','goto_from','goto_dest'
1,79fe2526bb9262f9ae1c9a30f1b4add61a480a31,/courses/course-v1:ComputerScience+MMDS+SelfPa...,USA,2016-12-19 20:40:49.584852,winter2016,course-v1:ComputerScience+MMDS+SelfPaced,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1
2,79fe2526bb9262f9ae1c9a30f1b4add61a480a31,/courses/course-v1:ComputerScience+MMDS+SelfPa...,USA,2016-12-19 20:40:50.625417,winter2016,course-v1:ComputerScience+MMDS+SelfPaced,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1
3,79fe2526bb9262f9ae1c9a30f1b4add61a480a31,/courses/course-v1:ComputerScience+MMDS+SelfPa...,USA,2016-12-19 20:40:52.075514,winter2016,course-v1:ComputerScience+MMDS+SelfPaced,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1
4,79fe2526bb9262f9ae1c9a30f1b4add61a480a31,/courses/course-v1:ComputerScience+MMDS+SelfPa...,USA,2016-12-19 20:40:55.819474,winter2016,course-v1:ComputerScience+MMDS+SelfPaced,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1
5,79fe2526bb9262f9ae1c9a30f1b4add61a480a31,page_close,USA,2016-12-19 20:40:58.173126,winter2016,course-v1:ComputerScience+MMDS+SelfPaced,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1
6,79fe2526bb9262f9ae1c9a30f1b4add61a480a31,/courses/course-v1:ComputerScience+MMDS+SelfPa...,USA,2016-12-19 20:40:59.244999,winter2016,course-v1:ComputerScience+MMDS+SelfPaced,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1
7,79fe2526bb9262f9ae1c9a30f1b4add61a480a31,/courses/course-v1:ComputerScience+MMDS+SelfPa...,USA,2016-12-19 20:41:00.245997,winter2016,course-v1:ComputerScience+MMDS+SelfPaced,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1
8,79fe2526bb9262f9ae1c9a30f1b4add61a480a31,/courses/course-v1:ComputerScience+MMDS+SelfPa...,USA,2016-12-19 20:41:01.250670,winter2016,course-v1:ComputerScience+MMDS+SelfPaced,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1
9,79fe2526bb9262f9ae1c9a30f1b4add61a480a31,/courses/course-v1:ComputerScience+MMDS+SelfPa...,USA,2016-12-19 20:41:02.246326,winter2016,course-v1:ComputerScience+MMDS+SelfPaced,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1


In [12]:
mmds_one_platform = mmds_one_platform[pd.isnull(mmds_one_platform.video_code)]
mmds_two_platform = mmds_two_platform[pd.isnull(mmds_two_platform.video_code)]

In [190]:
print mmds_one_platform.shape
print mmds_two_platform.shape
mmds_one_platform.sample(n=5)

(435980, 18)
(516837, 18)


,anon_screen_name,event_type,ip_country,time,quarter,course_display_name,resource_display_name,success,video_code,video_current_time,video_speed,video_old_time,video_new_time,video_seek_type,video_new_speed,video_old_speed,goto_from,goto_dest
384776,4eddaaac364f215247663374076ae720b1e27c9f,/courses/course-v1:ComputerScience+MMDS+SelfPa...,DEU,2017-03-20 13:46:49.096564,spring2017,course-v1:ComputerScience+MMDS+SelfPaced,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1
315111,c419a952a8170ad3408014b3835d985e36790b1b,/courses/course-v1:ComputerScience+MMDS+SelfPa...,USA,2017-03-12 03:57:24.759339,spring2017,course-v1:ComputerScience+MMDS+SelfPaced,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1
177147,a3c4b0e0eb4e62b28d9b9d2462bc54da2ded952a,/courses/course-v1:ComputerScience+MMDS+SelfPa...,GBR,2017-02-21 02:42:53.104300,winter2017,course-v1:ComputerScience+MMDS+SelfPaced,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1
737455,d3286f9e658275ce432c52c6c8e164a566714e4b,/courses/course-v1:ComputerScience+MMDS+SelfPa...,IND,2017-07-30 07:01:12.724100,summer2017,course-v1:ComputerScience+MMDS+SelfPaced,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1
270095,db4474006014bf38f5d1ee10209c5f1a80c46316,/courses/course-v1:ComputerScience+MMDS+SelfPa...,ITA,2017-03-10 08:42:54.326773,spring2017,course-v1:ComputerScience+MMDS+SelfPaced,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1


 * We have removed those actions that are video related. Now, let's merge the two tables.
 * After that we will see two ways of using these data.
 >  1. Just counting the number of times a user interacted with the platform components.
 >  2. Being granular and finding out the different kinds of platformn interaction
      and counting those.

In [191]:
mmds_platform = mmds_one_platform.append(mmds_two_platform, ignore_index=True)

In [192]:
print mmds_platform.shape
uniq_platform = mmds_platform.groupby(['anon_screen_name'])

(952817, 18)


In [193]:
print uniq_platform.size().shape
uniq_platform.size().sample(n=5)

(15635,)


anon_screen_name
3d08cc149772f0643623c85b298cb7f108b64dd7      3
fc00d7a7ae599e126763519ec1a626c60f4267af      4
5f3ef67ae7c32ecaa0b69a7a750e9bda0b920b99    706
7655d880f419708eeecf63b44ba34e7014977346      2
aca2d70537a87f18be83edd3011e80b3f9738539      1
dtype: int64

* So we have $15635$ unique non-video interactions, $4744$ unique video interactions and $505$
  `ActivityGrade` actions.
* The disparity between the video and non-video interactions could be explained by the 
  observation that some students simply download the videos and slides and view them
  offline. Their video actions are thus not recorded.
* In any case, we now have data to build our baseline model. But before that, lets have a 
  granular look at the non-video interactions.

In [194]:
granular_platform = mmds_platform.groupby(['anon_screen_name', 'event_type'])

In [36]:
granular_platform.size().sample(n=10)

* This is a mess. to really know the available unique events, lets save this data offline
  and inspect it with `regexes`, after that we can come up with a way of 
  categorizing the event types.

In [196]:
with open('event_types.txt', 'wb') as f:
    s = set(mmds_platform['event_type'])
    for l in s:
        f.write(str(l)+'\n')

* Exploring the different event_types reveals the following classes
> 1. Discussion
> 2. Download
> 3. Problem
> 4. `save_user_state` -- closely associated with video
> 5. Courseware

### 4. Model Building
The stage is set. Here is the plan for this section We create datasets with the following structures:

$1.$ Basic:

> |n-video-interactions | n-platform-interactions|avg-n-q-attempts|avg-grade|
|-----------------------|--------------------------|------------------|---------|
|$x_1$|$x_2$|$x_3$|$Y$|

$2.$ Granular Video

> |n-pauses|n-plays|n-seeks|n-stops|n-speed-change|n-platform-interactions|avg-n-q-attepts|avg-grade
|-----------------------|--------------------------|------------------|---------|
|$x_1$|$x_2$|$x_3$|$x_4$|$x_5$|$x_6$|$x_7$|$Y$

$3.$ Granular Platform interaction, non-granulated video

> |n-video_ints|n-discussion|n-download|n-problem|n-save_user_state|n-courseware|avg-n-q-attepts|avg-grade
|-----------------------|--------------------------|------------------|---------|
|$x_1$|$x_2$|$x_3$|$x_4$|$x_5$|$x_6$|$x_7$|$Y$

$4.$ Granular Everything

> |n-pauses|n-plays|n-seeks|n-stops|n-speed-change|n-discussion|n-download|n-problem|n-save_user_state|n-courseware|avg-n-q-attepts|avg-grade
|-----------------------|--------------------------|------------------|---------|
|$x_1$|$x_2$|$x_3$|$x_4$|$x_5$|$x_6$|$x_7$|$x_8$|$x_9$|$x_{10}$|$x_{11}$|$Y$

* Lets write code to generate these tables

In [8]:
courses = ['algorithms', 'mmds', 'compilers', 'stats']
tables = ['_activity_grade', '_platform_events', '_video_events']

def generate_activity_grade_agg(course):
    file_name = './Data/ActivityGradeTables/' + course + tables[0]
    data = read_csv(file_name +'.csv')
    data = data[data.num_attempts >= 0]
    grouped = data.groupby('anon_screen_name').agg({
        'num_attempts':np.mean, 
        'percent_grade':np.sum
    }).compute()
    grouped.to_csv(file_name + '_agg.csv')
    
for course in courses: 
    generate_activity_grade_agg(course) 

In [3]:
def generate_video_events_agg(course):
    file_name = './Data/' + course + tables[2]
    data = read_csv(file_name + '.csv')
    granular = data.groupby(['anon_screen_name', 'event_type']).size().compute()
    granular.to_csv(file_name + '_agg.csv')
    
# for course in courses:
#     generate_video_events_agg(course)

In [4]:
def categorize_event(col):
    type_name = col['event_type']
    if 'discussion' in type_name: return 'discussion'
    if 'download' in type_name: return 'download'
    if 'problem' in type_name: return 'problem'
    if 'save_user_state' in type_name: return 'save_user_state'
    if 'translation' in type_name: return 'translation'
    return 'courseware'

def generate_platform_events_agg(course):
    file_name = './Data/' + course + tables[1]
    data = read_csv(file_name + '.csv')
    data = data[dd.Series.isnull(data.video_code)]
    data['event_category'] = data.apply(categorize_event, axis=1)
    granular = data.groupby(['anon_screen_name', 'event_category']).size().compute()
    granular.to_csv(file_name + '_agg.csv')
    
# for course in courses:
#     generate_platform_events_agg(course)

* As expected most of the values are null. We now need to impute them.
* kNN works better than traditional imputation methods but is expensive.
* bPCA 

In [23]:
def create_video_data(course):
    file_name = './Data/VideoEventsTables/' + course + tables[2] + '_agg'
    data = pd.read_csv(file_name+'.csv', names=['id', 'event_type', 'count'])
    prev_id = None
    '''[load_video, pause_video, play_video, seek_video, speed_change_video, stop_video]'''
    counts = [0]*7
    count_matrix = []
    for _, row in data.iterrows():
        if not prev_id or row['id'] == prev_id:
            cur_event = row['event_type']
            if cur_event == 'load_video':
                i = 0
            elif cur_event == 'pause_video':
                i = 1
            elif cur_event == 'play_video':
                i = 2
            elif cur_event == 'seek_video':
                i = 3
            elif cur_event == 'speed_change_video':
                i = 4
            else:
                i = 5
            counts[i] = row['count']
            prev_id = row['id']
        else:
            prev_id = None
            counts[6] = sum(counts)
            counts = [row['id']] + counts
            count_matrix.append(counts)
            counts = [0]*7

    nu = pd.DataFrame(
    count_matrix,
    columns=['id', 'load_video', 'pause_video', 'play_video',
           'seek_video', 'speed_change_video', 'stop_video', 'total'
          ]
    )
    nu.to_csv(file_name+'_TABLE.csv')
    
# for course in courses:
#     create_video_data(course)

In [35]:
def create_platform_data(course):
    file_name = './Data/' + course + tables[1] + '_agg'
    data = pd.read_csv(file_name+'.csv', names=['id', 'event_type', 'count'])
    prev_id = None
    '''
     [courseware',
     'discussion',
     'download',
     'problem',
     'save_user_state',
     'translation']
    '''
    counts = [0]*7
    count_matrix = []
    for _, row in data.iterrows():
        if not prev_id or row['id'] == prev_id:
            cur_event = row['event_type']
            if cur_event == 'courseware':
                i = 0
            elif cur_event == 'discussion':
                i = 1
            elif cur_event == 'download':
                i = 2
            elif cur_event == 'problem':
                i = 3
            elif cur_event == 'save_user_state':
                i = 4
            else:
                i = 5
            counts[i] = row['count']
            prev_id = row['id']
        else:
            prev_id = None
            counts[6] = sum(counts)
            counts = [row['id']] + counts
            count_matrix.append(counts)
            counts = [0]*7

    nu = pd.DataFrame(
    count_matrix,
    columns=['id', 'courseware', 'discussion', 'download',
           'problem', 'save_user_state', 'translation', 'total'
          ]
    )
    nu.to_csv(file_name+'_TABLE.csv')
    
# for course in courses:
#     create_platform_data(course)